# Task

Берем отзывы за лето (из архива с материалами или предыдущего занятия)

Учим conv сеть для классификации
Рассмотреть 2-а варианта сеточек:
Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

# Preparing

In [23]:
!pip install -q stop_words
!pip install -q pymorphy2

In [48]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re
import gensim
import keras
import keras.backend as K
from sklearn.model_selection import train_test_split
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, GlobalMaxPool1D
from keras.callbacks import EarlyStopping, TensorBoard

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load Data

In [25]:
df = pd.read_excel('summer_review.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [26]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [27]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=13)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [28]:
df_train.head(3)

,Rating,Content,Date
0,5,отличное приложение,2017-07-31
1,1,Забанили телефон мейзу 6 про с оригинальной пр...,2017-08-01
2,5,ДОСТАТОЧНО БЫСТРО И ЛЕГКО,2017-08-13


# Preprocessing

In [29]:
sw = set(get_stop_words('ru'))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

Готовим словарь

In [30]:
train_corpus = " ".join(df_train['Content'])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [31]:
# Отфильтруем и возьмём только топ-N токенов
tokens_filtered = [word for word in tokens if word.isalnum()]

# Создаём словарь токен:количество в корпусе
dist = nltk.FreqDist(tokens_filtered)

In [32]:
max_words = 200
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words - 1 )]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [33]:
# Создаём словарь из топ-200 токенов. Токен:его рейтинг
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [34]:
max_len = 40

# Приводим к нижнему регистру, токенизируем, удаляем не цифры и не буквы.
# Проверяем, если токен есть в словаре, то в результат пишем его номер из словаря,
# в противном случае - пропускаем, до полной длины добиваем нулями
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0] * (maxlen - len(result))
    return padding + result[-maxlen:]

In [35]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_train

array([[  0,   0,   0, ...,   0,   8,   1],
       [  0,   0,   0, ..., 149,  23,  41],
       [  0,   0,   0, ...,   0,  11, 108],
       ...,
       [  0,   0,   0, ...,  62,  45,  53],
       [  0,   0,   0, ...,  26,  72,  13],
       [  0,   0,   0, ..., 126, 153,  18]], dtype=int32)

Для подсчета функции потерь y_train и y_test должны быть представлены one-hot кодированием

In [36]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating'])
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [37]:
train_enc_labels

array([4, 0, 4, ..., 0, 1, 4])

In [38]:
num_classes = 5
y_train = keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

# Keras CONV model with Embedding layer by default

In [45]:
model = Sequential()
# inputdim - размер словаря, outputdim - длина вектора,
# input_length - длина входной последовательности
# на вход: (батч, inputlen), на выходе: (батч, inputlen, outputdim)
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
# 128 - длина 1D-фильтра, шаг - 3
# на выходе ([128/3], 128)
model.add(Conv1D(128, 3))
# применяем функцию активации к выходу предыдущего слоя
# на выходе ([128/3], 128)
model.add(Activation("relu"))
# в каждой свертке оставляет максимальный элемент
# на выходе ([128/3], 1)
model.add(GlobalMaxPool1D())
# 10-количество выходов
model.add(Dense(10))
# num_classes = 5 -- количество выходов
model.add(Activation("relu"))
model.add(Dense(num_classes))
# преобразуем вектор в рапределение вероятностей
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 128)           25600     
                                                                 
 conv1d_2 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 activation_6 (Activation)   (None, 38, 128)           0         
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
 activation_7 (Activation)   (None, 10)                0         
                                                      

In [40]:
def get_f1(y_true, y_pred):
    """Функция подсчета f1_score"""
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [41]:
# y_pred должен быть распределением вероятностей,
# y_true - one-hot кодированный тензор
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [43]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping = EarlyStopping(monitor='val_loss')

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 7s 180ms/step - loss: 1.2085 - get_f1: 0.3383 - val_loss: 0.9876 - val_get_f1: 0.7043
Epoch 2/20
25/25 [==============================] - 7s 282ms/step - loss: 0.8740 - get_f1: 0.7333 - val_loss: 0.7802 - val_get_f1: 0.7462
Epoch 3/20
25/25 [==============================] - 4s 166ms/step - loss: 0.7439 - get_f1: 0.7523 - val_loss: 0.7177 - val_get_f1: 0.7523
Epoch 4/20
25/25 [==============================] - 5s 185ms/step - loss: 0.7124 - get_f1: 0.7558 - val_loss: 0.7033 - val_get_f1: 0.7524
Epoch 5/20
25/25 [==============================] - 5s 194ms/step - loss: 0.7005 - get_f1: 0.7590 - val_loss: 0.6994 - val_get_f1: 0.7542
Epoch 6/20
25/25 [==============================] - 4s 155ms/step - loss: 0.6928 - get_f1: 0.7603 - val_loss: 0.6932 - val_get_f1: 0.7525
Epoch 7/20
25/25 [==============================] - 5s 188ms/step - loss: 0.6860 - get_f1: 0.7604 - val_loss: 0.6882 - val_get_f1: 0.7516
Epoch 8/20
25/25 [================

In [44]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 1s 45ms/step - loss: 0.6997 - get_f1: 0.7501


Test loss: 0.699695348739624
Test f1_score: 0.7501370310783386


# Keras CONV model with pre-trained Embedding layer

In [46]:
!wget http://vectors.nlpl.eu/repository/20/180.zip

--2023-09-04 13:30:32--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  18.2MB/s    in 27s     

2023-09-04 13:31:00 (17.4 MB/s) - ‘180.zip’ saved [484452317/484452317]



In [47]:
!unzip 180.zip

Archive:  180.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [50]:
#Загрузим предобученные векторы
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('model.bin',
                                                               binary=True) .vectors

In [ ]:
#обрежем word_vectors по размеру эмбеддингов предыдущей задачи
word_vectors_matrix = [word_vectors[i][:128] for i in range(200)]
word_vectors_matrix

In [52]:
#инициализируем веса в эмбеддинге
initializer = keras.initializers.Constant(word_vectors_matrix)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128,
                    embeddings_initializer =initializer, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 40, 128)           25600     
                                                                 
 conv1d_3 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 activation_9 (Activation)   (None, 38, 128)           0         
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 10)                1290      
                                                                 
 activation_10 (Activation)  (None, 10)                0         
                                                      

In [53]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [54]:
tensorboard=TensorBoard(log_dir='logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 6s 169ms/step - loss: 1.0382 - get_f1: 0.5876 - val_loss: 0.8261 - val_get_f1: 0.7229
Epoch 2/20
25/25 [==============================] - 5s 223ms/step - loss: 0.7853 - get_f1: 0.7359 - val_loss: 0.7651 - val_get_f1: 0.7382
Epoch 3/20
25/25 [==============================] - 4s 156ms/step - loss: 0.7350 - get_f1: 0.7459 - val_loss: 0.7337 - val_get_f1: 0.7419
Epoch 4/20
25/25 [==============================] - 4s 157ms/step - loss: 0.7030 - get_f1: 0.7535 - val_loss: 0.7175 - val_get_f1: 0.7466
Epoch 5/20
25/25 [==============================] - 6s 224ms/step - loss: 0.6782 - get_f1: 0.7662 - val_loss: 0.7056 - val_get_f1: 0.7521
Epoch 6/20
25/25 [==============================] - 4s 161ms/step - loss: 0.6573 - get_f1: 0.7707 - val_loss: 0.6980 - val_get_f1: 0.7577
Epoch 7/20
25/25 [==============================] - 4s 158ms/step - loss: 0.6360 - get_f1: 0.7808 - val_loss: 0.7021 - val_get_f1: 0.7553


In [55]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 1s 43ms/step - loss: 0.7321 - get_f1: 0.7567


Test loss: 0.732106626033783
Test f1_score: 0.7567223310470581


Обе сети на метрике показали очень близкие результаты. F1-мера 0.7501 и 0.7567